# Zadanie domowe

W przypadku obrazów w odcieniach szarości pojedynczy piksel z zakresu [0; 255] reprezentowany jest jako 8-bitowa liczba bez znaku.
Pewnym rozszerzeniem analizy sposobu reprezentacji obrazu może być następujący eksperyment.
Załóżmy, że z każdego z 8 bitów możemy stworzyć pojedynczy obraz binarny (ang. _bit-plane slicing_).
Dla obrazka _100zloty.jpg_ (https://raw.githubusercontent.com/vision-agh/poc_sw/master/02_Point/100zloty.jpg) stwórz 8 obrazów, z których każdy powinien zawierać jedną płaszczyznę bitową.
Podpowiedź $-$ warto sprawdzić, jak realizuje się bitowe operacje logiczne.
Zastosowanie takiej dekompozycji obrazu pozwala na analizę ,,ważności'' poszczególnych bitów.
Jest to użyteczne w kwantyzacji, ale także w kompresji.

W drugim etapie zadania proszę spróbować odtworzyć obraz oryginalny z mniejszej liczby obrazów binarnych.
Warto zacząć od dwóch najbardziej znaczących bitów, a później dodawać kolejne.
Należy utworzyć co najmniej trzy wersje zrekonstruowanych obrazów.
Podpowiedź $-$ rekonstrukcja obrazu to mnożenie przez odpowiednią potęgę liczby 2 (przesunięcie bitowe) oraz dodawanie.

In [ ]:
import cv2
import os
import requests
import matplotlib.pyplot as plt
import numpy as np

url = 'https://raw.githubusercontent.com/vision-agh/poc_sw/master/02_Point/'
fileName = '100zloty.jpg'
if not os.path.exists(fileName) :
    r = requests.get(url + fileName, allow_redirects=True)
    open(fileName, 'wb').write(r.content)

In [ ]:
money = cv2.imread('100zloty.jpg')
money = cv2.cvtColor(money, cv2.COLOR_BGR2GRAY)
print(money.shape)

def plot_image(image, title="", cmap="gray"):
    plt.imshow(image, cmap=cmap)
    plt.title(title)
    plt.axis('off')
    plt.show()

def subplot_images(images, rows, cols, titles="", cmaps="gray", **kwargs):
    n = len(images)
    if not isinstance(titles, list):
        titles = [titles] * n
    if not isinstance(cmaps, list):
        cmaps = [cmaps] * n

    fig, axes = plt.subplots(rows, cols, **kwargs)
    if rows == 1:
        axes = [axes]
    for row in range(rows):
        for col in range(cols):
            idx = row * cols + col
            axes[row][col].imshow(images[idx], cmap=cmaps[idx])
            axes[row][col].set_title(titles[idx])
            axes[row][col].axis('off')
    plt.show()

plot_image(money)

In [ ]:
bit_partitions = []

for i in range(8):
    bit = 2 ** i
    and_image = cv2.bitwise_and(money, bit) / bit
    subplot_images((money, and_image), 1, 2, cmaps="gray", figsize=(10, 20))

    bit_partitions.append(and_image)

In [ ]:
leading_bit_img = cv2.add(bit_partitions[-1] * 128, bit_partitions[-2] * 64).astype(np.uint8)
planes = 2
mktitles = lambda num: [f"{num - 1}a: Original", f"{num - 1}b: merged {num} leading bit planes", f"Subtraction {num - 1}a - {num - 1}b"]

cmap = "inferno"

figsize = 15, 18

subtracted = cv2.subtract(money, leading_bit_img).astype(np.uint8)
subplot_images((money, leading_bit_img, subtracted), 1, 3, cmaps=cmap, titles=mktitles(planes), figsize=figsize)

for i in range(5, 1, -1):
    planes += 1
    leading_bit_img = cv2.add(leading_bit_img, np.multiply(bit_partitions[i], 2 ** i).astype(np.uint8)).astype(np.uint8)
    subtracted = cv2.subtract(money, leading_bit_img)
    subplot_images((money, leading_bit_img, subtracted), 1, 3, cmaps=cmap, titles=mktitles(planes), figsize=figsize)